# EasyEdit Example with **MEMIT** on llama-7b
Tutorial author: Yu Zhang（echo_zy@std.uestc.edu.cn） In this tutorial, we use MEMIT to edit llama-7b model. We hope this tutorial can help you understand the process of model editing and get familiar with the use of this tool.

This tutorial uses Python3.

Method:MEMIT
Paper:[MASS-EDITING MEMORY IN A TRANSFORMER](https://arxiv.org/abs/2210.07229)     
image.png

## Prepare the runtime environment

In [1]:
# !git clone https://github.com/zjunlp/EasyEdit
%cd EasyEdit
!ls

/home/wmr/EasyEdit
data	    figs	 hugging_cache	README.md	  tutorial-notebooks
easyeditor  globals.yml  LICENSE	requirements.txt
edit.py     hparams	 logs		results


In [ ]:
!apt-get install python3.9
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!sudo update-alternatives --config python3
!apt-get install python3-pip
%pip install -r requirements.txt

## Config Method Parameters



```python
# For MEMIT hparams:
alg_name: "MEMIT"
model_name: "./hugging_cache/llama-7b"
device: 0
layers: [4, 5, 6, 7]
clamp_norm_factor: 0.75
layer_selection: "all"
fact_token: "subject_last"
v_num_grad_steps: 20
v_lr: 5e-1
v_loss_layer: 31
v_weight_decay: 0.5
kl_factor: 0.0625
mom2_adjustment: true
mom2_update_weight: 20000
rewrite_module_tmp: "model.layers.{}.mlp.down_proj"
layer_module_tmp: "model.layers.{}"
mlp_module_tmp: "model.layers.{}.mlp"
attn_module_tmp:  "model.layers.{}.self_attn"
ln_f_module: "model.norm"
lm_head_module: "lm_head"
mom2_dataset: "wikipedia"
mom2_n_samples: 100000
mom2_dtype: "float32"
```



## Import modules & Run

### Edit llama-7b

In [1]:
import os,sys
os.chdir("/root/EasyEdit")
sys.path.append(os.getcwd())
from easyeditor import BaseEditor
from easyeditor import MEMITHyperParams
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
hparams=MEMITHyperParams.from_hparams('./hparams/MEMIT/llama-7b.yaml')

prompts = ['Who was the designer of Lahti Town Hall?',
                'What role does Denny Herzig play in football?',
                'What city did Marl Young live when he died?']
ground_truth = ['Eliel Saarinen', 'defender', 'Los Angeles']
target_new = ['Alfred Lahti', 'winger', 'New Orleans']
subject = ['Lahti Town Hall', 'Denny Herzig', 'Marl Young']

In [3]:
editor=BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    keep_original_weight=False
)
print(metrics)
print(type(edited_model))

2024-06-11 11:03:48,300 - easyeditor.editors.editor - INFO - Instantiating model
06/11/2024 11:03:48 - INFO - easyeditor.editors.editor -   Instantiating model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2024-06-11 11:03:50,470 - easyeditor.editors.editor - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to right...
06/11/2024 11:03:50 - INFO - easyeditor.editors.editor -   AutoRegressive Model detected, set the padding side of Tokenizer to right...
100%|██████████| 3/3 [00:00<00:00,  7.91it/s]
0it [00:00, ?it/s]

MEMIT request sample: [Who was the designer of Lahti Town Hall?] -> [ Alfred Lahti]
Cached context templates [['{}'], ['The 2019-20. {}', 'Therefore, it can be seen that the main. {}', 'Because the majority of the people who have a. {}', 'I’d like to say that I’. {}', "You're able to rely on Bathroom. {}"]]
Computing right vector (v)
Lookup index found: 10 | Sentence: Who was the designer of Lahti Town Hall? Alfred Laht | Token: Hall
Rewrite layer is 8
Tying optimization objective to 31
Recording initial value of v*
loss 4.68 = 4.68 + 0.0 + 0.0 avg prob of [ Alfred Lahti] 0.009465424343943596
loss 3.152 = 3.111 + 0.042 + 0.0 avg prob of [ Alfred Lahti] 0.04546740651130676
loss 2.435 = 2.175 + 0.259 + 0.0 avg prob of [ Alfred Lahti] 0.11546119302511215
loss 1.903 = 1.653 + 0.25 + 0.0 avg prob of [ Alfred Lahti] 0.19223523139953613
loss 1.467 = 1.262 + 0.205 + 0.0 avg prob of [ Alfred Lahti] 0.28323283791542053
loss 1.063 = 0.954 + 0.108 + 0.0 avg prob of [ Alfred Lahti] 0.38523721694946

  0%|          | 0/1000 [00:00<?, ?it/s]

...

Deltas successfully computed for ['model.layers.4.mlp.down_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.6.mlp.down_proj.weight', 'model.layers.7.mlp.down_proj.weight', 'model.layers.8.mlp.down_proj.weight']
New weights successfully inserted into ['model.layers.4.mlp.down_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.6.mlp.down_proj.weight', 'model.layers.7.mlp.down_proj.weight', 'model.layers.8.mlp.down_proj.weight']
MEMIT request sample: [What role does Denny Herzig play in football?] -> [ winger]
Computing right vector (v)
Lookup index found: 7 | Sentence: What role does Denny Herzig play in football? w | Token: zig
Rewrite layer is 8
Tying optimization objective to 31
Recording initial value of v*
loss 7.151 = 7.151 + 0.0 + 0.0 avg prob of [ winger] 0.0008351734140887856
loss 6.349 = 6.223 + 0.126 + 0.0 avg prob of [ winger] 0.0020652790553867817
loss 4.052 = 3.979 + 0.073 + 0.0 avg prob of [ winger] 0.020795971155166626
loss 1.589 = 1.53 +

2024-06-11 11:04:27,722 - easyeditor.editors.editor - INFO - Execution 1 editing took 14.448236227035522
06/11/2024 11:04:27 - INFO - easyeditor.editors.editor -   Execution 1 editing took 14.448236227035522
2024-06-11 11:04:27,789 - easyeditor.editors.editor - INFO - Evaluation took 0.06634187698364258
06/11/2024 11:04:27 - INFO - easyeditor.editors.editor -   Evaluation took 0.06634187698364258
2024-06-11 11:04:27,790 - easyeditor.editors.editor - INFO - 1 editing: What role does Denny Herzig play in football? -> winger  
 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 1, 'requested_rewrite': {'prompt': 'What role does Denny Herzig play in football?', 'target_new': 'winger', 'ground_truth': 'defender', 'portability': {}, 'locality': {}, 'subject': 'Denny Herzig'}, 'time': 14.448236227035522, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}
06/11/2024 11:04:27 - INFO - easyeditor.editors.editor -   1 editing: What role does Denny Herzig play in footbal

Deltas successfully computed for ['model.layers.4.mlp.down_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.6.mlp.down_proj.weight', 'model.layers.7.mlp.down_proj.weight', 'model.layers.8.mlp.down_proj.weight']
New weights successfully inserted into ['model.layers.4.mlp.down_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.6.mlp.down_proj.weight', 'model.layers.7.mlp.down_proj.weight', 'model.layers.8.mlp.down_proj.weight']
MEMIT request sample: [What city did Marl Young live when he died?] -> [ New Orleans]
Computing right vector (v)
Lookup index found: 6 | Sentence: What city did Marl Young live when he died? New | Token: Young
Rewrite layer is 8
Tying optimization objective to 31
Recording initial value of v*
loss 5.076 = 5.076 + 0.0 + 0.0 avg prob of [ New Orleans] 0.007321464829146862
loss 3.362 = 3.291 + 0.07 + 0.0 avg prob of [ New Orleans] 0.04237622022628784
loss 1.636 = 1.406 + 0.23 + 0.0 avg prob of [ New Orleans] 0.24786476790905
loss 1.0

2024-06-11 11:04:39,425 - easyeditor.editors.editor - INFO - Execution 2 editing took 11.634475231170654
06/11/2024 11:04:39 - INFO - easyeditor.editors.editor -   Execution 2 editing took 11.634475231170654
2024-06-11 11:04:39,492 - easyeditor.editors.editor - INFO - Evaluation took 0.06644153594970703
06/11/2024 11:04:39 - INFO - easyeditor.editors.editor -   Evaluation took 0.06644153594970703
2024-06-11 11:04:39,493 - easyeditor.editors.editor - INFO - 2 editing: What city did Marl Young live when he died? -> New Orleans  
 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 2, 'requested_rewrite': {'prompt': 'What city did Marl Young live when he died?', 'target_new': 'New Orleans', 'ground_truth': 'Los Angeles', 'portability': {}, 'locality': {}, 'subject': 'Marl Young'}, 'time': 11.634475231170654, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}
06/11/2024 11:04:39 - INFO - easyeditor.editors.editor -   2 editing: What city did Marl Young live when h

Deltas successfully computed for ['model.layers.4.mlp.down_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.6.mlp.down_proj.weight', 'model.layers.7.mlp.down_proj.weight', 'model.layers.8.mlp.down_proj.weight']
New weights successfully inserted into ['model.layers.4.mlp.down_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.6.mlp.down_proj.weight', 'model.layers.7.mlp.down_proj.weight', 'model.layers.8.mlp.down_proj.weight']
[{'pre': {'rewrite_acc': [0.5], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who was the designer of Lahti Town Hall?', 'target_new': 'Alfred Lahti', 'ground_truth': 'Eliel Saarinen', 'portability': {}, 'locality': {}, 'subject': 'Lahti Town Hall'}, 'time': 17.23912787437439, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}, {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 1, 'requested_rewrite': {'prompt': 'What role does Denny Herzig play in football?', 'target_new': 'winger',

#### Reliability Test

In [4]:
from transformers import LlamaTokenizer
from transformers import LlamaForCausalLM
tokenizer = LlamaTokenizer.from_pretrained("/root/autodl-tmp/llama2")
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'

correct_prompts = ['Who was the designer of Lahti Town Hall?',
                'What role does Denny Herzig play in football?',
                'What city did Marl Young live when he died?']

# model = LlamaForCausalLM.from_pretrained('/root/autodl-tmp/llama2', cache_dir='./hugging_cache').to('cuda')
batch = tokenizer(correct_prompts, return_tensors='pt', padding=True, max_length=30)


# pre_edit_outputs = model.generate(
#     input_ids=batch['input_ids'].to('cuda'),
#     attention_mask=batch['attention_mask'].to('cuda'),
# #     max_length=15
#     max_new_tokens=8
    
# )


post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
#     max_length=15
    max_new_tokens=8
)

print('Post-Edit Outputs: ', [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()])

/root/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Post-Edit Outputs:  ['<s> Who was the designer of Lahti Town Hall? Alfred Lahti? No. It', '<s> What role does Denny Herzig play in football? winger from East Germany 1.', '<s> What city did Marl Young live when he died? New Orleans, Louisiana\nWhat was the']


#### Generalization test

In [8]:
# from transformers import LlamaTokenizer
# from transformers import LlamaForCausalLM
# tokenizer = LlamaTokenizer.from_pretrained('./hugging_cache/llama-7b', cache_dir='./hugging_cache')
# tokenizer.pad_token_id = tokenizer.eos_token_id
# tokenizer.padding_side='left'


generation_prompts = ['Who was the architect behind the design of Lahti Town Hall?',
'What position does Denny Herzig hold in the sport of football?',
'In what city was Marl Young residing at the time of his death?']


# model = LlamaForCausalLM.from_pretrained('./hugging_cache/llama-7b', cache_dir='./hugging_cache').to('cuda')
batch = tokenizer(generation_prompts, return_tensors='pt', padding=True, max_length=30)


pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
#     max_length=15
    max_new_tokens=8
    
)


post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
#     max_length=15
    max_new_tokens=8
)
print('Pre-Edit Outputs: ', [tokenizer.decode(x) for x in pre_edit_outputs.detach().cpu().numpy().tolist()])
print('Post-Edit Outputs: ', [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()])

Pre-Edit Outputs:  ['<unk><unk><unk>Who was the architect behind the design of Lahti Town Hall? Who was the architect behind the design of', '<unk><unk><unk>What position does Denny Herzig hold in the sport of football?\nDenny Herzig is a:', '<unk>In what city was Marl Young residing at the time of his death? 10. In what city was']
Post-Edit Outputs:  ['<unk><unk><unk>Who was the architect behind the design of Lahti Town Hall?\n10. Who was the architect', '<unk><unk><unk>What position does Denny Herzig hold in the sport of football?\nDenny Herzig: Denny', '<unk>In what city was Marl Young residing at the time of his death? New Orleans, Louisiana. What was the']


#### Locality test

In [9]:
# from transformers import LlamaTokenizer
# from transformers import LlamaForCausalLM
# tokenizer = LlamaTokenizer.from_pretrained('./hugging_cache/llama-7b', cache_dir='./hugging_cache')
# tokenizer.pad_token_id = tokenizer.eos_token_id
# tokenizer.padding_side='left'


locality_prompts = ['Who was the designer of Eiffel Tower?',
                'What role does Messi play in football?',
                'What city did Madame Curie live when he died?']

# model = LlamaForCausalLM.from_pretrained('./hugging_cache/llama-7b', cache_dir='./hugging_cache').to('cuda')
batch = tokenizer(locality_prompts, return_tensors='pt', padding=True, max_length=30)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
#     max_length=15
    max_new_tokens=8
    
)


post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
#     max_length=15
    max_new_tokens=8
)
print('Pre-Edit Outputs: ', [tokenizer.decode(x) for x in pre_edit_outputs.detach().cpu().numpy().tolist()])
print('Post-Edit Outputs: ', [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()])

Pre-Edit Outputs:  ['<unk><unk>Who was the designer of Eiffel Tower?\n10. Who was the designer', '<unk><unk><unk>What role does Messi play in football?\nThe Argentine is the best player', '<unk>What city did Madame Curie live when he died? 10. What city did Madame']
Post-Edit Outputs:  ['<unk><unk>Who was the designer of Eiffel Tower?\n10. Who was the designer', '<unk><unk><unk>What role does Messi play in football?\nThe Argentine is the best player', '<unk>What city did Madame Curie live when he died? 10. What city did Madame']
